## Random Forest Model
- This model got 0.86662 (top 76%). This serves as a baseline / benchmark for my other models

## XGBoost
- This model got 0.87546 (top 71%). This is a slight improvement on my random forest model

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc

# Load the data
data = pd.read_csv('data/train.csv')
X = pd.get_dummies(data.drop(['Exited', 'CustomerId', 'Surname'], axis=1), columns=['Geography', 'Gender'])
y = data['Exited']

# Make train-dev-test split (80-10-10)
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2)
test_data = pd.read_csv('data/test.csv')
X_test = pd.get_dummies(test_data.drop(['CustomerId', 'Surname'], axis=1), columns=['Geography', 'Gender'])

In [3]:
# Train a Random Forest Classifier
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [4]:
# Make predicitons using train split
preds = random_forest.predict_proba(X_dev)[:, 1]
fpr, tpr, thresholds = roc_curve(y_dev, preds)
roc_auc = auc(fpr, tpr)
print(f"{roc_auc=}")

roc_auc=0.8777890639535744


In [5]:
import xgboost as xgb

# Train an XGBoost model

xgb_model = xgb.XGBClassifier(objective='binary:logistic')
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:
# Make predictions
preds = xgb_model.predict_proba(X_dev)[:, 1]
fpr, tpr, thresholds = roc_curve(y_dev, preds)
roc_auc = auc(fpr, tpr)
print(f"{roc_auc=}")

roc_auc=0.8878144486060164


In [10]:
final_xgb = xgb.XGBClassifier(objective='binary:logistic') 
final_xgb.fit(X, y)
final_xgb_preds = final_xgb.predict_proba(X_test)[:,1]

In [11]:
# Make submission file for xgb
submission = pd.DataFrame({'id': X_test['id'],
                           'Exited': final_xgb_preds
                          })

submission.to_csv('xgboost.csv', index=False)

In [13]:
# Train model again using all data
final_random_forest = RandomForestClassifier()
final_random_forest.fit(X, y)
final_preds = final_random_forest.predict_proba(X_test)[:, 1]

In [12]:
# Make submission file for rf
submission = pd.DataFrame({'id': X_test['id'],
                           'Exited': final_preds
                          })

submission.to_csv('random_forest.csv', index=False)

NameError: name 'final_preds' is not defined